In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data_clear.csv')
data.sample(n=10)

,wallsMaterial,floorNumber,floorsTotal,totalArea,kitchenArea,latitude,longitude,price,distance,azimuth
53153,panel,16,17.0,55.0,9.2,55.840866,37.383894,9500000,17568.0,304.0
39360,panel,9,9.0,33.1,7.0,55.687000,37.720933,6700000,9824.0,138.0
48026,block,1,5.0,78.0,8.7,55.689623,37.564033,13900000,7684.0,205.0
9093,panel,11,17.0,37.0,9.0,55.862040,37.555696,7990000,12793.0,343.0
15567,block,8,12.0,45.0,5.8,55.715778,37.810575,7400000,12902.0,108.0
19413,block,2,5.0,34.0,5.2,55.648844,37.579313,7000000,11731.0,191.0
38976,panel,9,12.0,94.4,11.0,55.715043,37.894173,14700000,17983.0,103.0
9965,panel,8,17.0,58.1,9.0,55.778310,37.526618,14950000,6297.0,298.0
36702,block,5,12.0,35.4,10.0,55.750439,37.811222,6350000,12285.0,91.0
43458,panel,9,12.0,45.4,6.5,55.613255,37.603711,8300000,15488.0,183.0


# Разделяем данные и создаем модель

In [3]:
y = data['price']
features = [
            'wallsMaterial', 
            'floorNumber', 
            'floorsTotal', 
            'totalArea', 
            'kitchenArea',
            'distance',
            'azimuth'
           ]
categorical_features = ['wallsMaterial']
X = data[features]
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=1)

train_pool = Pool(train_X, 
                  train_y, 
                  cat_features=categorical_features)
test_pool = Pool(test_X, 
                 cat_features=categorical_features)

In [99]:
grid = {
        'iterations': [2000, 2500, 3000, 3500, 4000, 5000],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 10]
       }

model = CatBoostRegressor(loss_function='MAE')
grid_search_result = model.grid_search(grid, train_pool, plot=True)
grid_search_result

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 1804897.915
bestIteration = 1999

0:	loss: 1804897.9150798	best: 1804897.9150798 (0)	total: 19.7s	remaining: 15m 24s

bestTest = 1700337.226
bestIteration = 1999

1:	loss: 1700337.2255203	best: 1700337.2255203 (1)	total: 38.1s	remaining: 14m 36s

bestTest = 1778086.022
bestIteration = 2499

2:	loss: 1778086.0220012	best: 1700337.2255203 (1)	total: 1m 5s	remaining: 16m 27s

bestTest = 1685741.705
bestIteration = 2499

3:	loss: 1685741.7050331	best: 1685741.7050331 (3)	total: 1m 30s	remaining: 16m 34s

bestTest = 1757947.16
bestIteration = 2999

4:	loss: 1757947.1600299	best: 1685741.7050331 (3)	total: 2m	remaining: 17m 20s

bestTest = 1676376.196
bestIteration = 2999

5:	loss: 1676376.1964522	best: 1676376.1964522 (5)	total: 2m 35s	remaining: 18m 5s

bestTest = 1743447.515
bestIteration = 3499

6:	loss: 1743447.5148166	best: 1676376.1964522 (5)	total: 3m 9s	remaining: 18m 27s

bestTest = 1668030.32
bestIteration = 3499

7:	loss: 1668030.3200157	best: 1668030.3200157 (7)	tota

{'params': {'depth': 10, 'iterations': 5000, 'learning_rate': 0.03},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
 

In [107]:
preds = model.predict(test_pool)
print('Mean: ', (abs(preds - test_y) / test_y * 100).mean())
print('Median: ', (abs(preds - test_y) / test_y * 100).median())
print('Max: ', (abs(preds - test_y) / test_y * 100).max())
print('Min: ', (abs(preds - test_y) / test_y * 100).min())

Mean:  8.795207330317632
Median:  6.3753092850824835
Max:  298.2859965343854
Min:  0.0015877743029007406


In [4]:
result_pool = Pool(X, 
                  y, 
                  cat_features=categorical_features)
model = CatBoostRegressor(depth=10, iterations=5000, learning_rate=0.03, loss_function='MAE')
model.fit(result_pool, verbose=False)

In [6]:
model.save_model('apartments_model')